# Project 1: Trends in General Happiness versus Financial Satisfaction and Work Satisfaction Over Time
## Group 11
- Elizabeth Hunter (mwm6nq)
- Michael Hijduk (aqt7bt)
- Eva Massarelli (ecm8yu)
- Anjali Mehta (wfn7ad)
- Anika Potluri (agu4yh)
## ! git clone https://www.github.com/DS3001/group11

# Summary
Research question(s), overview of methods and findings. Make sure to reference the codebook and describe the research strategy...

# Data
Describe the source of data, variables, and challenges weaving in the cleaning code as is relevant. 

Create wrangling and exploratory data analysis as subsections here...

In [ ]:
import pandas as pd

df = pd.read_csv('./data/selectedData.csv')

# Results
Show visualizations and discuss findings...

# Conclusion
Re-summarize the project, defend from criticism, suggest future work...